In [1]:
import sys

In [5]:
!{sys.executable} -m pip install numpy==1.26.4
!{sys.executable} -m pip install unsloth vllm==0.7.2
!{sys.executable} -m pip install -U ipywidgets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.2 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [2]:
# from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import re
from datasets import load_dataset

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-07 22:31:58 __init__.py:190] Automatically detected platform cuda.


In [6]:
SYSTEM_PROMPT = '''
You are playing the NY Times Connections game. Your task is to categorize 16 given words into exactly 4 groups of 4 words each, based on shared common themes.

Solve the puzzle using these clear steps:

1. THINK STEP-BY-STEP: Begin by carefully analyzing the words within <thinking> tags. Identify their meanings, relationships, and possible groupings logically.
Example:
<thinking>To solve this, I will first look for obvious groupings like NBA teams, palindrome words, weather-related words, or computer keys. Then, I'll group these words accordingly and ensure each word belongs to exactly one group.</thinking>

2. SHOW YOUR REASONING: After identifying each grouping, briefly explain why these words belong together. Clearly indicate the context or theme.

3. PROVIDE FINAL ANSWER: After clearly grouping and justifying all four sets, provide ONLY your final solution within <answer> tags. Format your solution exactly as shown below.

Example:
<answer>
[['HAIL', 'RAIN', 'SLEET', 'SNOW'],
 ['BUCKS', 'HEAT', 'JAZZ', 'NETS'],
 ['OPTION', 'RETURN', 'SHIFT', 'TAB'],
 ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']]
</answer>

Important Notes:
- Categories should be specific
- Words cannot appear in more than one group.
- Categories can include compound words, shared prefixes/suffixes, pop culture references, or common phrases.

Here is an example:

USER: [BUCKS, HAIL, JAZZ, SHIFT, LEVEL, MOM, SNOW, RACECAR, SLEET, TAB, KAYAK, RETURN, OPTION, NETS, RAIN, HEAT]

SOLUTION:
[['HAIL', 'RAIN', 'SLEET', 'SNOW'],
 ['BUCKS', 'HEAT', 'JAZZ', 'NETS'],
 ['OPTION', 'RETURN', 'SHIFT', 'TAB'],
 ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']]

Explanation:
- WEATHER TERMS: 'HAIL', 'RAIN', 'SLEET', 'SNOW'
- NBA TEAMS: 'BUCKS', 'HEAT', 'JAZZ', 'NETS'
- KEYBOARD KEYS: 'OPTION', 'RETURN', 'SHIFT', 'TAB'
- PALINDROMES: 'KAYAK', 'LEVEL', 'MOM', 'RACECAR'
'''

In [7]:
def get_connections_questions(split = "train"):
    data = load_dataset('csv', data_files='dataset/final_transformed_connections.csv')[split]
    data = data.train_test_split(test_size=0.1, seed=3407)
    train_data = data['train']
    test_data = data['test']
    train_data = train_data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'assistant', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['questions']}
        ],
        'answer': (x['answer'])
    })
    return train_data, test_data # type: ignore

dataset, test_dataset = get_connections_questions()

Map:   0%|          | 0/580 [00:00<?, ? examples/s]

In [8]:
def thinking_reward_func(completions, **kwargs) -> list[float]:
    """Reward function for including thinking tags"""
    rewards = []
    for completion in completions:
        try:
            reward = 0.0
            # Extract all thinking blocks
            for message in completion:
                if message["role"] == "assistant" and message.get("content"):
                    content = message["content"]

                    # Count opening and closing tags
                    opening_tags = len(re.findall(r"<thinking>", content))
                    closing_tags = len(re.findall(r"</thinking>", content))

                    if opening_tags == 0 or closing_tags == 0:
                        continue

                    if opening_tags == closing_tags:
                        reward += 0.5
                    else:
                        reward += 0.1
            reward = min(reward, 1.5)
            rewards.append(reward)
        except Exception as e:
            print(f"{RED}Error in thinking_reward_func: {e}{RESET}")
            rewards.append(0.0)
    assert len(rewards) == len(completions)
    return rewards


def answer_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function for including answer tags"""
    rewards = []
    for completion in completions:
        try:
            reward = 0.0
            # Extract all answer blocks
            for message in completion:
                if message["role"] == "assistant" and message.get("content"):
                    content = message["content"]

                    # Count opening and closing tags
                    opening_tags = len(re.findall(r"<answer>", content))
                    closing_tags = len(re.findall(r"</answer>", content))

                    if opening_tags == 0 or closing_tags == 0:
                        continue

                    if opening_tags == 1 and closing_tags == 1:
                        reward += 0.5
                    else:
                        reward += 0.1
            reward = min(reward, 1.5)
            rewards.append(reward)
        except Exception as e:
            print(f"{RED}Error in thinking_reward_func: {e}{RESET}")
            rewards.append(0.0)
    assert len(rewards) == len(completions)
    return rewards


def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """Reward function tailored for NY Times Connections game puzzle answers"""
    rewards = []
    for completion, expected_answer in zip(completions, answer):
        try:
            reward = 0.0

            # Extract the answer from completion
            predicted_answer = None
            for message in completion:
                if message["role"] == "assistant":
                    content = message.get("content", "")
                    matches = re.findall(r"<answer>(.*?)</answer>", content, re.DOTALL)
                    if matches:
                        predicted_answer = matches[-1].strip()

            if predicted_answer is None:
                rewards.append(0.0)
                continue

            # Convert string representations of lists into actual Python lists for comparison
            try:
                predicted_groups = eval(predicted_answer)
                expected_groups = eval(expected_answer)

                if isinstance(predicted_groups, list) and isinstance(expected_groups, list):
                    if predicted_groups == expected_groups:
                        reward += 5.0
                    else:
                        # Check partial matches (correct groups)
                        correct_groups = sum(1 for group in predicted_groups if group in expected_groups)
                        reward += correct_groups * 1.25  # Partial credit per correct group
                else:
                    reward = 0.0

            except Exception:
                reward = 0.0

            rewards.append(reward)

        except Exception as e:
            print(f"Error in correctness_reward_func: {e}")
            rewards.append(0.0)

    assert len(rewards) == len(completions)
    return rewards

In [9]:
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 8 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.0. vLLM: 0.7.2.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.451 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-1.5b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.66%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 44.45 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 20.86 GB. Also swap space = 6 GB.
INFO 04-07 22:33:06 config.py:542] This model supports multiple tasks: {'classify', 'embed', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config us

[W407 22:33:07.662663999 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 04-07 22:33:08 loader.py:1102] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 04-07 22:33:09 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-07 22:33:42 model_runner.py:1115] Loading model weights took 1.4331 GB
INFO 04-07 22:33:42 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-07 22:33:46 worker.py:267] Memory profiling takes 4.17 seconds
INFO 04-07 22:33:46 worker.py:267] the current vLLM instance can use total_gpu_memory (44.45GiB) x gpu_memory_utilization (0.50) = 22.08GiB
INFO 04-07 22:33:46 worker.py:267] model weights take 1.43GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.57GiB; the rest of the memory reserved for KV Cache is 19.02GiB.
INFO 04-07 22:33:46 executor_base.py:110] # CUDA blocks: 44509, # CPU blocks: 14043
INFO 04-07 22:33:46 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 695.45x
INFO 04-07 22:33:50 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory erro

Capturing CUDA graph shapes: 100%|██████████| 39/39 [00:26<00:00,  1.45it/s]

INFO 04-07 22:34:17 model_runner.py:1562] Graph capturing finished in 27 secs, took 0.67 GiB
INFO 04-07 22:34:17 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 35.32 seconds



Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 16, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = 128,
    max_completion_length = 64,
    num_train_epochs = 1, # Set to 1 for a full training run
    save_steps = 10,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [11]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        thinking_reward_func,
        correctness_reward_func,
        answer_format_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 580 | Num Epochs = 1 | Total steps = 36
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 16 x 1) = 64
 "-____-"     Trainable parameters = 9,232,384/5,000,000,000 (0.18% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / thinking_reward_func,rewards / correctness_reward_func,rewards / answer_format_reward_func
1,-0.000000,0.031250,0.062500,63.296875,0.000000,0.031250,0.000000,0.000000
2,-0.000000,0.023438,0.046875,63.328125,0.000000,0.007812,0.000000,0.015625
3,0.000000,0.015625,0.031250,64.000000,0.001225,0.007812,0.000000,0.007812
4,0.000000,0.007812,0.015625,64.000000,0.001231,0.007812,0.000000,0.000000
5,0.000100,0.023438,0.046875,63.671875,0.001289,0.015625,0.000000,0.007812
6,0.000000,0.023438,0.046875,63.921875,0.001176,0.015625,0.000000,0.007812
7,0.000000,0.023438,0.046875,64.000000,0.001151,0.015625,0.000000,0.007812
8,0.000100,0.023438,0.046875,64.000000,0.001284,0.015625,0.000000,0.007812
9,0.000000,0.007812,0.015625,64.000000,0.001146,0.000000,0.000000,0.007812
10,0.000000,0.000000,0.000000,64.000000,0.001182,0.000000,0.000000,0.000000


TrainOutput(global_step=36, training_loss=4.649792238827407e-05, metrics={'train_runtime': 911.0325, 'train_samples_per_second': 0.637, 'train_steps_per_second': 0.04, 'total_flos': 0.0, 'train_loss': 4.649792238827407e-05})

In [12]:
model.save_lora("grpo_saved_lora")

In [15]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "[BOXER, SLOW, MUZZLE, DALMATIAN, HUSKY, WAVE, SNOUT, DETOUR, TRUNK, YIELD, ASLEEP, BARRIER, POODLE, BEAK, STOP, BITE]"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.21it/s, est. speed input: 701.44 toks/s, output: 116.10 toks/s]


"<thinking>To solve this, I will first look for obvious groupings like animals, objects, related words, or common themes.</thinking>\n\n<answer>\n[['BITE', 'YIELD', 'STOP', 'BEAK'],\n ['HUSKY', 'DALMATIAN', 'WAVE', 'POODLE'],\n ['BOXER', 'WAVE', 'BEAK'],\n ['YIELD', 'ASLEEP', 'BITE', 'STOP']]\n</answer>"

In [14]:
test_dataset[0]

{'questions': 'BOXER, SLOW, MUZZLE, DALMATIAN, HUSKY, WAVE, SNOUT, DETOUR, TRUNK, YIELD, ASLEEP, BARRIER, POODLE, BEAK, STOP, BITE',
 'answer': "[['BOXER', 'DALMATIAN', 'HUSKY', 'POODLE'], ['BEAK', 'MUZZLE', 'SNOUT', 'TRUNK'], ['DETOUR', 'SLOW', 'STOP', 'YIELD'], ['ASLEEP', 'BARRIER', 'BITE', 'WAVE']]",
 'grouped_answers': "DOG BREEDS - ['BOXER', 'DALMATIAN', 'HUSKY', 'POODLE']\nANIMAL NOSES - ['BEAK', 'MUZZLE', 'SNOUT', 'TRUNK']\nTRAFFIC SIGNS - ['DETOUR', 'SLOW', 'STOP', 'YIELD']\nSOUND ___ - ['ASLEEP', 'BARRIER', 'BITE', 'WAVE']"}